## prep scriptbase

In [6]:
import os
import tarfile
inputdir1 = './scriptbase/scriptbase_j/'
inputdir2 = './scriptbase_extracted/'

fault = []

for filename in os.listdir(inputdir1):
    # print(filename)
    try:
        file = tarfile.open(inputdir1 + filename, 'r:gz')
        # print(file.getnames())
        file.extract('./'+ filename[:-7] + '/processed/profile.txt', inputdir2)
        file.extract('./'+ filename[:-7] + '/processed/script_clean.txt', inputdir2)
        file.close()
        # with open(inputdir2 + filename[:-7] + '/processed/profile.txt', 'r', encoding='UTF-8') as f:
        #     text = f.read()
        #     print(text)
        # with open(inputdir2 + filename[:-7] + '/processed/script_clean.txt', 'r', encoding='UTF-8') as f:
        #     text = f.read()
        #     print(text)
    except:
        fault.append(filename)
        print(f'{filename} produced an error.')

1492 produced an error.
A Nightmare on Elm Street 2 produced an error.
A Nightmare on Elm Street 3 produced an error.
A Nightmare on Elm Street 4 produced an error.
A Nightmare on Elm Street 5 produced an error.
Airplane II produced an error.
Austin Powers produced an error.
Blade produced an error.
Book of Shadows produced an error.
Freddys Dead produced an error.
Friday the 13th Part VI produced an error.
Friday the 13th Part VIII produced an error.
Halloween produced an error.
Halloween 4 produced an error.
Halloween H20 produced an error.
Hellbound produced an error.
Hellraiser produced an error.
Hellraiser III produced an error.
Highlander III produced an error.
Interview with the Vampire produced an error.
Jaws produced an error.
Lara Croft produced an error.
Mighty Morphin Power Rangers produced an error.
Mission produced an error.
Rambo produced an error.
Robin Hood produced an error.
South Park produced an error.
Star Trek produced an error.
Star Trek II produced an error.
Sta

In [7]:
print(fault)

for filename in fault[:10]:
    file = tarfile.open(inputdir1 + filename, 'r:gz')
    print(file.getnames())

['1492', 'A Nightmare on Elm Street 2', 'A Nightmare on Elm Street 3', 'A Nightmare on Elm Street 4', 'A Nightmare on Elm Street 5', 'Airplane II', 'Austin Powers', 'Blade', 'Book of Shadows', 'Freddys Dead', 'Friday the 13th Part VI', 'Friday the 13th Part VIII', 'Halloween', 'Halloween 4', 'Halloween H20', 'Hellbound', 'Hellraiser', 'Hellraiser III', 'Highlander III', 'Interview with the Vampire', 'Jaws', 'Lara Croft', 'Mighty Morphin Power Rangers', 'Mission', 'Rambo', 'Robin Hood', 'South Park', 'Star Trek', 'Star Trek II', 'Star Trek V', 'Star Trek VI', 'Star Wars Episode I', 'Star Wars Episode II', 'Star Wars Episode III', 'Superman IV', 'Sweeney Todd', 'Terminator 2', 'The Boondock Saints II', 'The Crow', 'The Lord of the Rings', 'The Lost World', 'The Messenger', 'The Transformers', 'The Twilight Saga', 'Tron', 'Twin Peaks', 'Wild Things', 'X-Men Origins']


ReadError: empty file

In [12]:
import re
import spacy
import numpy as np
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
allowed = set('ADJ NOUN VERB'.split())

def remove_brack(string):
    pattern = re.compile("[\(\[].*?[\)\]]")
    string = re.sub(pattern, "", string)
    return string

def get_spoken(lines):
    if len(lines[0])-len(lines[0].lstrip()) != 0:
        white = int((len(lines[0])-len(lines[0].lstrip()))*1.5)
    else:
        white = 5
    pattern = re.compile('\s{'+ f'{white}' + '}')
    spoken_text = ''
    for line in lines:
        line = remove_brack(line)
        if re.match(pattern, line) is not None and line.isupper() is False:
            spoken_text += line.strip() + ' '
    return spoken_text



def prep_spoken(string):
    toklst = []
    string = string.casefold()
    tokens = nlp(string)
    if tokens:
        for t in tokens:
            if t.pos_ in allowed and t.is_alpha and not t.is_stop:
                toklst.append(t.text)
    return ' '.join(toklst).casefold()

outputdir = './scriptbase_texts/'

for subdir in [f for f in os.listdir(inputdir2)]:
    # print(subdir)
    with open(inputdir2 + subdir + '/processed/script_clean.txt', 'r', encoding='UTF-8') as f:
        lines = f.readlines()
        spoken_text = get_spoken(lines)
        if len(spoken_text) == 0:
            print(f'No text in {subdir}')
        spoken_prepped = prep_spoken(spoken_text)
    with open(outputdir + subdir + '.txt', 'w', encoding='UTF-8') as f:
        f.write(spoken_prepped)

In [84]:
dct = {}

from string import punctuation

for subdir in [f for f in os.listdir(inputdir2)]:
    # print(subdir)
    with open(inputdir2 + subdir + '/processed/profile.txt', 'r', encoding='UTF-8') as f:
        lines = f.readlines()
        pattern1 = re.compile(r'.+main genres* (are|is)')
        pattern2 = re.compile(r'.+is a.+movie')
        for line in lines:
            if re.match(pattern1, line) is not None:
                genreline = re.sub(pattern1, '', line)
                genreline = re.sub(r'[,\s]and', '', genreline)
                genreline = genreline[:-2]
                genres = genreline.split(',')
                genres = [genre.split() for genre in genres]
                genres = [genre for genreplus in genres for genre in genreplus]
                dct[subdir] = [genre.strip() for genre in genres]
            elif re.match(pattern2, line) is not None:
                genreline = re.sub(r'.+is a[n\s]', '', line)
                genreline = re.sub(r'movie.', '', genreline)
                genreline = re.sub(r'[,\s]and', '', genreline)
                genres = genreline.split(',')
                genres = [genre.split() for genre in genres]
                genres = [genre for genreplus in genres for genre in genreplus]
                dct[subdir] = [genre.strip() for genre in genres]
            else:
                pass

In [85]:
lst = [item for value in dct.values() for item in value]
print(set(lst))
no = ['features','is', 'involves', 'has', 'a', 'structure','in', 'film']

for key, value in dct.items():
    for item in value:
        if item in no:
            dct[key] = [i for i in value if i != item]

{'family', 'historical', 'independent', 'romance', 'comedy', 'animation', 'adventure', 'war', 'mockumentary', 'sci-fi', 'action', 'arts', 'period', 'blaxpoitation', 'drama', 'noir', 'features', 'horror', 'crime', 'fantasy', 'is', 'mystery', 'martial', 'involves', 'parody', 'musical', 'western', 'has', 'expressionist', 'foreign', 'a', 'road', 'thriller', 'sports', 'structure', 'talky.', 'in', 'film', 'music-oriented', 'erotic'}


In [86]:
lst = [item for value in dct.values() for item in value]
print(set(lst))

{'family', 'historical', 'independent', 'romance', 'comedy', 'animation', 'adventure', 'war', 'mockumentary', 'sci-fi', 'action', 'arts', 'period', 'blaxpoitation', 'drama', 'noir', 'horror', 'crime', 'fantasy', 'is', 'mystery', 'martial', 'parody', 'musical', 'western', 'has', 'expressionist', 'foreign', 'a', 'road', 'thriller', 'sports', 'structure', 'talky.', 'in', 'film', 'music-oriented', 'erotic'}


In [89]:
import json

with open('scriptbase_meta.json', 'w', encoding ='utf8') as json_file:
    json.dump(dct, json_file, ensure_ascii = False)